In [0]:
%%shell

rm -rf ./torch-scae
git clone https://github.com/bdsaglam/torch-scae
cd ./torch-scae
pip install -q .

In [0]:
%load_ext tensorboard

!mkdir /content/lightning_logs
%tensorboard --logdir '/content/lightning_logs'

In [0]:
import pathlib

import numpy as np
import torch
import torchvision
from torch_scae_experiments.mnist.train import train
from torch_scae_experiments.mnist.hparams import model_params
from pytorch_lightning import seed_everything

seed_everything(42)

In [0]:
from pprint import pprint

pprint(model_params)

In [0]:
# Customize model parameters

# model_params.update(dict(
#     n_part_caps=36,
#     n_obj_caps=10,
#     pcae_template_generator_params=dict(
#         template_size=(11, 11),
#         template_nonlin='sigmoid',
#         colorize_templates=False,
#     ),
#     scae_params=dict(
#         posterior_sparsity_loss_type='entropy',
#         posterior_within_example_sparsity_weight=1,
#         posterior_between_example_sparsity_weight=1,
#     )
# ))

In [0]:
training_params = dict(
    data_dir=str(pathlib.Path.home() / 'torch-datasets'),
    gpus=1,
    batch_size=128,
    num_workers=2,
    max_epochs=100,
    learning_rate=3e-5,
    optimizer_type='RMSprop',
    use_lr_scheduler=True,
    lr_scheduler_decay_rate=0.997
)

In [3]:
import pathlib
import hydra

hydra._internal.hydra.GlobalHydra().clear()

config_dir = pathlib.Path('./torch-scae/torch-scae-experiments/configs/config.yaml')
hydra.experimental.initialize(config_dir=config_dir)

cfg = hydra.experimental.compose(config_file='config.yaml', overrides=[])
print(cfg.pretty())

AttributeError: module 'hydra' has no attribute 'initialize'

Object `hydra.compose` not found.


In [4]:
import torch_scae

In [9]:
import torch_scae_experiments

In [6]:
import pathlib

In [10]:
directory = pathlib.Path(torch_scae_experiments.__file__).parent

In [11]:
list(directory.iterdir())

[PosixPath('/Users/bdsaglam/anaconda3/envs/ml/lib/python3.7/site-packages/torch_scae_experiments/mnist'),
 PosixPath('/Users/bdsaglam/anaconda3/envs/ml/lib/python3.7/site-packages/torch_scae_experiments/__init__.py'),
 PosixPath('/Users/bdsaglam/anaconda3/envs/ml/lib/python3.7/site-packages/torch_scae_experiments/__pycache__')]

In [ ]:
from torch_scae_experiments.mnist.train import train

In [0]:
torch.cuda.reset_max_memory_cached()
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_accumulated_memory_stats()

train(model_params, **training_params)

In [0]:
# resume training
training_params.update(
    max_epochs=200,
    resume_from_checkpoint= '/content/lightning_logs/version_0/checkpoints/epoch=99.ckpt')
)
train(model_params, **training_params)